#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

<b> 논문: "Attention Is All You Need"

프롬프트 주제:
<li> 프롬프트1. Transformer가 딥러닝과 다르게 생성형 모델로 작동하는 이유
<li> 프롬프트2. Transformer가 멀티모달 인간 모방 트렌드를 위한 것이라는 논리
<li> 프롬프트3. 딥러닝 vs Transformer – 뇌 vs 언어습득 방식 모방
<li> 각 프롬프트는 Chain of Thought (CoT) → Self-Consistency (SC) → ReAct 흐름에 맞게 구성 <b>

<b>📘 프롬프트 1 (CoT → SC → ReAct)
<li> Transformer 모델은 여전히 딥러닝 프레임워크에서 학습되지만, 그 구조는 순환이나 컨볼루션 없이 동작하며 실제로는 생성형 모델처럼 기능합니다.
<li> 이러한 구조적 특성이 왜 딥러닝의 연산 방식과 다른 방향으로 작동하게 되는지 단계별로 설명해 보세요. ← 🧠 CoT
<li> 그런 다음, 이 구조가 다른 딥러닝 모델들과 비교해 생성적 추론 능력에서 어떤 정합성을 갖는지 평가해보세요. ← 🔍 SC
<li> 마지막으로, Transformer의 구조를 다른 생성형 아키텍처(예: GAN, VAE)와 융합한다면 어떤 결과가 나올지 시뮬레이션 해보세요. ← 🛠 ReAct <b>

<b>📘 프롬프트 2 (CoT → SC → ReAct)
<li> Transformer 구조는 언어, 이미지, 코드 등 다양한 모달리티에 동일하게 적용되는 특성이 있습니다.
<li> 왜 이런 구조가 인간의 멀티모달 처리 방식을 모방했다고 볼 수 있는지 단계적으로 추론해 보세요. ← 🧠 CoT
<li> 이어서, CNN, RNN 등 기존 아키텍처와 비교하여 Transformer가 멀티모달 통합에 있어 더 정합적인 이유를 평가해보세요. ← 🔍 SC
<li> 마지막으로, Transformer가 실제 인간의 감각 처리 흐름(예: 시각 + 언어 통합)을 모방하는 가상의 시나리오를 구성해보세요. ← 🛠 ReAct </b>

<b> 📘 프롬프트 3 (CoT → SC → ReAct)
<li> 딥러닝은 주로 뇌의 신경망 구조를 흉내내는 데서 시작되었지만, Transformer는 인간의 언어 습득 과정을 모방한 것이라는 주장이 있습니다.
<li> 이 두 가지 철학적 차이를 구체적으로 비교 분석하며 단계적으로 설명해 보세요. ← 🧠 CoT
<li> 실제 학습 방식이나 데이터 사용 관점에서 이 차이가 어떤 정합성을 갖는지도 논리적으로 검토해보세요. ← 🔍 SC
<li> 마지막으로, Transformer가 아이가 언어를 익히는 방식과 유사하게 학습되는 사례를 시뮬레이션해 보세요. ← 🛠 ReAct </b>

In [4]:
%pip install langchain-core langchain-community langchain-openai faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
#논문 로딩
from pypdf import PdfReader

def load_paper(path):
    reader = PdfReader(path)
    return "\n".join([p.extract_text() for p in reader.pages if p.extract_text()])

paper_text = load_paper("paper-attention.pdf")

In [ ]:
#문서 분할 & 벡터 저장
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.create_documents([paper_text])

# .env 파일 불러오기
from dotenv import load_dotenv
load_dotenv()

# 환경변수에서 API 키 읽기
import os
api_key = os.getenv("OPENAI_API_KEY")

vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings(api_key=api_key))

In [7]:
#프롬프트 목록(한글)
prompts = [
    """Transformer 모델은 여전히 딥러닝 프레임워크에서 학습되지만, 그 구조는 순환이나 컨볼루션 없이 동작하며 실제로는 생성형 모델처럼 기능합니다.
이러한 구조적 특성이 왜 딥러닝의 연산 방식과 다른 방향으로 작동하게 되는지 단계별로 설명해 보세요.
그런 다음, 이 구조가 다른 딥러닝 모델들과 비교해 생성적 추론 능력에서 어떤 정합성을 갖는지 평가해보세요.
마지막으로, Transformer의 구조를 다른 생성형 아키텍처(예: GAN, VAE)와 융합한다면 어떤 결과가 나올지 시뮬레이션 해보세요.""",

    """Transformer 구조는 언어, 이미지, 코드 등 다양한 모달리티에 동일하게 적용되는 특성이 있습니다.
왜 이런 구조가 인간의 멀티모달 처리 방식을 모방했다고 볼 수 있는지 단계적으로 추론해 보세요.
이어서, CNN, RNN 등 기존 아키텍처와 비교하여 Transformer가 멀티모달 통합에 있어 더 정합적인 이유를 평가해보세요.
마지막으로, Transformer가 실제 인간의 감각 처리 흐름(예: 시각 + 언어 통합)을 모방하는 가상의 시나리오를 구성해보세요.""",

    """딥러닝은 주로 뇌의 신경망 구조를 흉내내는 데서 시작되었지만, Transformer는 인간의 언어 습득 과정을 모방한 것이라는 주장이 있습니다.
이 두 가지 철학적 차이를 구체적으로 비교 분석하며 단계적으로 설명해 보세요.
실제 학습 방식이나 데이터 사용 관점에서 이 차이가 어떤 정합성을 갖는지도 논리적으로 검토해보세요.
마지막으로, Transformer가 아이가 언어를 익히는 방식과 유사하게 학습되는 사례를 시뮬레이션해 보세요."""
]

In [8]:
#LLM으로 답변 생성 함수
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

def answer_with_context(prompt, top_k=3):
    related_docs = vectorstore.similarity_search(prompt, k=top_k)
    context = "\n\n".join([doc.page_content for doc in related_docs])
    full_prompt = f"""너는 논문 기반 연구 보조 AI야.
다음은 논문에서 발췌한 내용이야:\n\n{context}\n\n이제 아래 복합 질문에 단계적으로 답변해줘:\n{prompt}"""
    
    llm = ChatOpenAI(model="gpt-4o", temperature=0.3)
    return llm.invoke([HumanMessage(content=full_prompt)]).content

In [9]:
#실행 및 결과 출력
for i, q in enumerate(prompts, 1):
    print(f"\n🧠 복합 프롬프트 {i}:\n{q}\n")
    response = answer_with_context(q)
    print(f"💬 GPT-4o 응답 {i}:\n{response}\n{'-'*80}")


🧠 복합 프롬프트 1:
Transformer 모델은 여전히 딥러닝 프레임워크에서 학습되지만, 그 구조는 순환이나 컨볼루션 없이 동작하며 실제로는 생성형 모델처럼 기능합니다.
이러한 구조적 특성이 왜 딥러닝의 연산 방식과 다른 방향으로 작동하게 되는지 단계별로 설명해 보세요.
그런 다음, 이 구조가 다른 딥러닝 모델들과 비교해 생성적 추론 능력에서 어떤 정합성을 갖는지 평가해보세요.
마지막으로, Transformer의 구조를 다른 생성형 아키텍처(예: GAN, VAE)와 융합한다면 어떤 결과가 나올지 시뮬레이션 해보세요.

💬 GPT-4o 응답 1:
Transformer 모델은 딥러닝 프레임워크에서 학습되지만, 전통적인 순환 신경망(RNN)이나 합성곱 신경망(CNN)과는 다른 방식으로 작동합니다. 이를 단계별로 설명하겠습니다.

### 단계 1: Transformer의 구조적 특성

1. **자기-어텐션(Self-Attention)**:
   - Transformer는 입력 시퀀스의 각 요소가 다른 모든 요소와의 관계를 학습할 수 있도록 자기-어텐션 메커니즘을 사용합니다. 이는 입력 시퀀스의 각 위치에서 다른 모든 위치의 중요도를 계산하여, 시퀀스 내의 모든 요소 간의 관계를 한 번에 파악할 수 있게 합니다.

2. **병렬 처리**:
   - RNN은 순차적으로 데이터를 처리해야 하므로 병렬화가 어렵습니다. 반면, Transformer는 자기-어텐션을 통해 모든 입력을 동시에 처리할 수 있어 병렬화가 용이합니다. 이는 학습 속도를 크게 향상시킵니다.

3. **포지셔널 인코딩(Positional Encoding)**:
   - 순환 구조가 없는 Transformer는 시퀀스의 순서를 인식하기 위해 포지셔널 인코딩을 사용합니다. 이는 각 입력 위치에 고유한 위치 정보를 추가하여 모델이 순서를 인식할 수 있도록 합니다.

### 단계 2: 생성적 추론 능력에서의 정합성

Transformer의 구조는 다음과 같은 이유로 생성적 추론에서 강력한 성능을 발휘합니다: